In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt

from numpy.random import seed

In [ ]:
train = pd.read_csv("/kaggle/input/dataquest2020/energy_train.csv")
train['degree_C1'] = train['degree_C1'].fillna(train['degree_C1'].mean())
train['degree_C3'] = train['degree_C1'].fillna(train['degree_C3'].mean())
train['moisture_9'] = train['moisture_9'].fillna(train['moisture_9'].mean())
train.isnull().sum()

In [ ]:
train['degree'] = train['degree_C1'] + train['degree_C3'] + train['degree_C2'] + train['degree_C4'] + train['degree_C5'] + train['degree_C6'] +train['degree_C7'] + train['degree_C8'] + train['degree_C9']
train = train.drop(columns=['degree_C2', 'degree_C4','degree_C1', 'degree_C3', 'degree_C5', 'degree_C6', 'degree_C7', 'degree_C8', 'degree_C9'])

train['moisture'] = train['moisture_1'] + train['moisture_2'] + train['moisture_3'] + train['moisture_4'] + train['moisture_5'] + train['moisture_6'] +train['moisture_7'] + train['moisture_8']
train = train.drop(columns=['moisture_1', 'moisture_2', 'moisture_3', 'moisture_4', 'moisture_5', 'moisture_6', 'moisture_7', 'moisture_8'])

train['random'] = train['random_variable_1'] + train['random_variable_2']
train = train.drop(columns=['random_variable_1', 'random_variable_2'])
train

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['degree_C1'], y = train['WattHour'])
plt.ylabel('luminousity', fontsize=13)
plt.xlabel('Watthour', fontsize=13)
plt.show()

In [ ]:
print(train.shape)

train.info()
train = train.sample(frac=1)
train.isnull().sum()

In [ ]:
train.columns

In [ ]:
continuous = ['luminousity', 'degree_C1', 'moisture_1',
       'degree_C2', 'moisture_2', 'degree_C3', 'moisture_3', 'degree_C4',
       'moisture_4', 'degree_C5', 'moisture_5', 'degree_C6', 'moisture_6',
       'degree_C7', 'moisture_7', 'degree_C8', 'moisture_8', 'degree_C9',
       'moisture_9', 'degree_Cout', 'Pressure', 'moisture_out', 'Wind',
       'Clarity', 'dew_index', 'random_variable_1', 'random_variable_2']

def norm(var):
    return ((var - var.mean())/var.std())

for var in continuous:
    train[var] = train[var].astype('float64')
    train[var] = norm(train[var])

train

In [ ]:
# Watthour is label
train_inputs = train.drop(columns=['WattHour', 'id', 'date'])
train_labels = train['WattHour']

train_len = 14200
dev_inputs = train_inputs[train_len:]
dev_labels = train_labels[train_len:]

train_inputs = train_inputs[:train_len]
train_labels = train_labels[:train_len]

print(train_inputs.shape)
print(train_labels.shape)
print(dev_inputs.shape)
print(dev_labels.shape)

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
display_all(train.describe(include='all').T)

In [ ]:
seed(42)
tf.random.set_seed(42)

model = tf.keras.Sequential([
	tf.keras.layers.Dense(units=64,  kernel_initializer='normal', kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='relu', input_shape=(27,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(units=100,  kernel_initializer='normal', kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
    tf.keras.layers.Dense(units=64,  kernel_initializer='normal', kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear'),
])

model.summary()

In [ ]:
#optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00038, beta_1=0.97, beta_2=0.999, amsgrad=False)
#optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001),
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

In [ ]:
history = model.fit(train_inputs, train_labels, epochs=100, batch_size=512, validation_data=(dev_inputs, dev_labels))

In [ ]:
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
test = pd.read_csv("/kaggle/input/dataquest2020/energy_test.csv")
test['moisture_9'] = test['moisture_9'].fillna(36)

test.isnull().sum()

In [ ]:
continuous = ['luminousity', 'degree_C1', 'moisture_1',
       'degree_C2', 'moisture_2', 'degree_C3', 'moisture_3', 'degree_C4',
       'moisture_4', 'degree_C5', 'moisture_5', 'degree_C6', 'moisture_6',
       'degree_C7', 'moisture_7', 'degree_C8', 'moisture_8', 'degree_C9',
       'moisture_9', 'degree_Cout', 'Pressure', 'moisture_out', 'Wind',
       'Clarity', 'dew_index', 'random_variable_1', 'random_variable_2']

scaler = StandardScaler()

for var in continuous:
    test[var] = test[var].astype('float64')
    test[var] = scaler.fit_transform(test[var].values.reshape(-1, 1))

test

In [ ]:
# Watthour is label
test_inputs = test.drop(columns=['id', 'date'])
# train_labels = train['WattHour']


print(test_inputs.shape)
test_inputs.columns

In [ ]:
ans = model.predict(test_inputs)

ans[ans < 0] = 0
        
ans_df = pd.DataFrame(ans, columns=['WattHour'])
print(ans_df.shape)
print(test['id'].shape)
pass_id = test['id']
result = pd.concat([pass_id, ans_df], axis=1, sort=False)
result

In [ ]:
result = result.round(0).astype(int)   
result

In [ ]:
result.to_csv('sample_submission.csv', index=False)